# Final Project. E-mobility in Germany

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import re

### Reading the charging station dataframe into pandas

In [2]:
# Loading the xls file into a pandas dataframe
eautos = pd.read_excel('Ladesaeulenregister.xlsx', skiprows=10)
eautos.head(20)

,Betreiber,Straße,Hausnummer,Adresszusatz,Postleitzahl,Ort,Bundesland,Kreis/kreisfreie Stadt,Breitengrad,Längengrad,...,Public Key1,Steckertypen2,P2 [kW],Public Key2,Steckertypen3,P3 [kW],Public Key3,Steckertypen4,P4 [kW],Public Key4
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,...,NaN,AC Steckdose Typ 2,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,smopi®,Albstraße,14,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,...,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22.0,NaN,AC Steckdose Typ 2,22,NaN
2,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,0,NaN,72589,Westerheim,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.5105,9.609,...,NaN,AC Steckdose Typ 2,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,NaN,73340,Amstetten,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.578534,9.87484,...,NaN,"DC Kupplung Combo, DC CHAdeMO",50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SWU Energie GmbH,Sterngasse,6,NaN,88481,Balzheim,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SWU Energie GmbH,Herwigstraße,21,NaN,89129,Langenau,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.48524,10.028291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,EnBW mobility+ AG und Co.KG,Flurstück,313,NaN,89129,Langenau/Seligweiler,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.456817,10.029649,...,NaN,DC Kupplung Combo,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,EnBW mobility+ AG und Co.KG,Flurstück,313,NaN,89129,Langenau/Seligweiler,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.456817,10.029649,...,NaN,DC Kupplung Combo,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,EnBW mobility+ AG und Co.KG,Flurstück,313,NaN,89129,Langenau/Seligweiler,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.456817,10.029649,...,NaN,AC Steckdose Typ 2,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,EnBW mobility+ AG und Co.KG,Flurstück,313,NaN,89129,Langenau/Seligweiler,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.456817,10.029649,...,NaN,AC Steckdose Typ 2,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
eautos.shape

(44227, 26)

### Translating columns into english 

In [4]:
old_names = eautos.columns
old_names

Index(['Betreiber', 'Straße', 'Hausnummer', 'Adresszusatz', 'Postleitzahl',
       'Ort', 'Bundesland', 'Kreis/kreisfreie Stadt', 'Breitengrad',
       'Längengrad', 'Inbetriebnahmedatum',
       'Nennleistung Ladeeinrichtung [kW]', 'Art der Ladeeinrichung',
       'Anzahl Ladepunkte', 'Steckertypen1', 'P1 [kW]', 'Public Key1',
       'Steckertypen2', 'P2 [kW]', 'Public Key2', 'Steckertypen3', 'P3 [kW]',
       'Public Key3', 'Steckertypen4', 'P4 [kW]', 'Public Key4'],
      dtype='object')

In [5]:
new_names = ['operator', 'street', 'haus_no', 'address_extension', 'postcode',
       'city', 'federal_land', 'county', 'latitude', 
       'longitude', 'commissioning_date', 
       'power_connection_[kw]', 'type_of_charger',
       'number_of_charging_points', 'type_of_plug_1', 'p1_[kw]', 'public_key1',
       'type_of_plug_2', 'p2_[kw]', 'public_key2', 'type_of_plug_3', 'p3_[kw]',
       'public_key3', 'type_of_plug_4', 'p4_[kw]', 'public_key4'
            ]

In [6]:
eautos = eautos.rename(columns=dict(zip(old_names, new_names)))

# check the translation
eautos.head(2)

,operator,street,haus_no,address_extension,postcode,city,federal_land,county,latitude,longitude,...,public_key1,type_of_plug_2,p2_[kw],public_key2,type_of_plug_3,p3_[kw],public_key3,type_of_plug_4,p4_[kw],public_key4
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,...,NaN,AC Steckdose Typ 2,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,smopi®,Albstraße,14,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,...,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22.0,NaN,AC Steckdose Typ 2,22,NaN


In [7]:
# Translating the valeus for the type of charger column
eautos['type_of_charger'].unique() # checking the values

array(['Normalladeeinrichtung', 'Schnellladeeinrichtung'], dtype=object)

In [8]:
#Actual translation
eautos.type_of_charger.replace({'Schnellladeeinrichtung': 'fast', 'Normalladeeinrichtung': 'normal'}, inplace=True)

In [9]:
eautos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44227 entries, 0 to 44226
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   operator                   44227 non-null  object        
 1   street                     44227 non-null  object        
 2   haus_no                    44227 non-null  object        
 3   address_extension          6292 non-null   object        
 4   postcode                   44227 non-null  int64         
 5   city                       44227 non-null  object        
 6   federal_land               44227 non-null  object        
 7   county                     44227 non-null  object        
 8   latitude                   44227 non-null  object        
 9   longitude                  44227 non-null  object        
 10  commissioning_date         44227 non-null  datetime64[ns]
 11  power_connection_[kw]      44227 non-null  object        
 12  type

In [10]:
columns_na = ['type_of_plug_2', 'p2_[kw]', 'type_of_plug_3', 'p3_[kw]', 'type_of_plug_4', 'p4_[kw]', 'haus_no']

for column in columns_na:
    eautos[column] = eautos[column].fillna(value='0')

### Changing the data types for some columns to Float 

In [11]:
columns_modify = ['longitude', 'latitude', 'power_connection_[kw]']

In [12]:
# As there are some values with "." as the separator and others have "," as a separtor, the first line of the loop is replacing only those that are "#"
for column in columns_modify:
    eautos.loc[eautos[column].str.contains(',', na=False), column] = eautos.loc[eautos[column].str.contains(',', na=False), column].str.replace(',', '.')
    eautos[column] = eautos[column].astype('float')

In [13]:
# double checking the results
eautos['power_connection_[kw]'].value_counts(dropna=False)

44.0     15001
22.0     11252
11.0      2486
3.7       1657
30.0      1645
         ...  
65.0         1
41.0         1
116.4        1
142.0        1
170.0        1
Name: power_connection_[kw], Length: 215, dtype: int64

### Checking for duplicates

In [14]:
eautos.duplicated().sum()

5810

In [15]:
pd.set_option('display.max_columns', None)
duplicate_rows  = eautos[eautos.duplicated(keep=False)]
duplicate_rows

,operator,street,haus_no,address_extension,postcode,city,federal_land,county,latitude,longitude,commissioning_date,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],public_key1,type_of_plug_2,p2_[kw],public_key2,type_of_plug_3,p3_[kw],public_key3,type_of_plug_4,p4_[kw],public_key4
6,EnBW mobility+ AG und Co.KG,Flurstück,313,NaN,89129,Langenau/Seligweiler,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.456817,10.029649,2019-03-08,300.0,fast,2,DC Kupplung Combo,300,NaN,DC Kupplung Combo,300,NaN,0,0,NaN,0,0,NaN
7,EnBW mobility+ AG und Co.KG,Flurstück,313,NaN,89129,Langenau/Seligweiler,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.456817,10.029649,2019-03-08,300.0,fast,2,DC Kupplung Combo,300,NaN,DC Kupplung Combo,300,NaN,0,0,NaN,0,0,NaN
81,Aral Pulse,Nellinger Straße,24,NaN,89188,Merklingen,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.516260,9.759398,2019-12-17,300.0,fast,2,"DC Kupplung Combo, DC CHAdeMO",300,NaN,DC Kupplung Combo,300,NaN,0,0,NaN,0,0,NaN
82,Aral Pulse,Nellinger Straße,24,NaN,89188,Merklingen,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.516260,9.759398,2019-12-17,300.0,fast,2,"DC Kupplung Combo, DC CHAdeMO",300,NaN,DC Kupplung Combo,300,NaN,0,0,NaN,0,0,NaN
166,EnBW mobility+ AG und Co.KG,Illertal West,0,BAB7,88451,Dettingen,Baden-Württemberg,Landkreis Biberach,48.125522,10.109017,2020-12-28,300.0,fast,2,DC Kupplung Combo,300,NaN,DC Kupplung Combo,300,NaN,0,0,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44205,Aral Pulse,Aralallee,1,NaN,99441,Mellingen,Thüringen,Landkreis Weimarer Land,50.934670,11.389635,2022-11-04,300.0,fast,2,"DC Kupplung Combo, DC CHAdeMO",300,NaN,DC Kupplung Combo,300,NaN,0,0,NaN,0,0,NaN
44218,EnBW mobility+ AG und Co.KG,Robert-Koch-Straße,10,NaN,99510,Apolda,Thüringen,Landkreis Weimarer Land,51.028940,11.517130,2023-03-01,150.0,fast,2,DC Kupplung Combo,150,NaN,DC Kupplung Combo,150,NaN,0,0,NaN,0,0,NaN
44219,EnBW mobility+ AG und Co.KG,Robert-Koch-Straße,10,NaN,99510,Apolda,Thüringen,Landkreis Weimarer Land,51.028940,11.517130,2023-03-01,150.0,fast,2,DC Kupplung Combo,150,NaN,DC Kupplung Combo,150,NaN,0,0,NaN,0,0,NaN
44220,EnBW mobility+ AG und Co.KG,Robert-Koch-Straße,10,NaN,99510,Apolda,Thüringen,Landkreis Weimarer Land,51.028940,11.517130,2023-03-01,300.0,fast,2,DC Kupplung Combo,300,NaN,DC Kupplung Combo,300,NaN,0,0,NaN,0,0,NaN


Trying to understand if those are real duplicates or just several enteries for the charding stations with more than 1 charging point.

In [16]:
duplicate_rows[duplicate_rows['number_of_charging_points'] > 2]

,operator,street,haus_no,address_extension,postcode,city,federal_land,county,latitude,longitude,commissioning_date,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],public_key1,type_of_plug_2,p2_[kw],public_key2,type_of_plug_3,p3_[kw],public_key3,type_of_plug_4,p4_[kw],public_key4
2234,smopi®,Wilhelm Maybach Straße,2,MLOG Logistics GmbH,74196,Neuenstadt am Kocher,Baden-Württemberg,Landkreis Heilbronn,49.234998,9.352034,2022-12-30,22.0,normal,4,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22.0,NaN,AC Steckdose Typ 2,22,NaN
2235,smopi®,Wilhelm Maybach Straße,2,MLOG Logistics GmbH,74196,Neuenstadt am Kocher,Baden-Württemberg,Landkreis Heilbronn,49.234998,9.352034,2022-12-30,22.0,normal,4,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22.0,NaN,AC Steckdose Typ 2,22,NaN
2304,Ladepark FISCHER-ZANDER Erlenbach,Käppelesäcker,10,NaN,74235,Erlenbach,Baden-Württemberg,Landkreis Heilbronn,49.167881,9.261240,2022-03-01,22.0,normal,4,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22.0,NaN,AC Steckdose Typ 2,22,NaN
2305,Ladepark FISCHER-ZANDER Erlenbach,Käppelesäcker,10,NaN,74235,Erlenbach,Baden-Württemberg,Landkreis Heilbronn,49.167881,9.261240,2022-03-01,22.0,normal,4,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22.0,NaN,AC Steckdose Typ 2,22,NaN
2323,EnBW mobility+ AG und Co.KG,A 81 / Am Autobahnkreuz,0,NaN,74248,Weinsberg (Ellhofen),Baden-Württemberg,Landkreis Heilbronn,49.143625,9.308848,2021-12-23,300.0,fast,3,DC Kupplung Combo,300,NaN,"DC Kupplung Combo, DC CHAdeMO",300,NaN,AC Steckdose Typ 2,22.0,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43790,Ladepark Ehrhardt AG Hildburghausen,Kaltenbronner Weg,2,VW,98646,Hildburghausen,Thüringen,Landkreis Hildburghausen,50.426500,10.725900,2022-07-01,300.0,fast,4,AC Kupplung Typ 2,22,NaN,DC Kupplung Combo,22,NaN,DC Kupplung Combo,150.0,NaN,DC Kupplung Combo,150,NaN
43979,Q1 Autostrom,Neustädter Straße,50,NaN,7819,Triptis,Thüringen,Landkreis Saale-Orla-Kreis,50.737564,11.845551,2021-05-10,150.0,fast,3,DC Kupplung Combo,150,NaN,DC Kupplung Combo,150,NaN,AC Steckdose Typ 2,22.0,NaN,0,0,NaN
43980,Q1 Autostrom,Neustädter Straße,50,NaN,7819,Triptis,Thüringen,Landkreis Saale-Orla-Kreis,50.737564,11.845551,2021-05-10,150.0,fast,3,DC Kupplung Combo,150,NaN,DC Kupplung Combo,150,NaN,AC Steckdose Typ 2,22.0,NaN,0,0,NaN
44094,EnBW mobility+ AG und Co.KG,Raststätte Leubinger Fürstenhügel,0,BAB71,99610,Sömmerda,Thüringen,Landkreis Sömmerda,51.188317,11.162712,2021-03-31,300.0,fast,3,DC Kupplung Combo,300,NaN,"DC Kupplung Combo, DC CHAdeMO",300,NaN,AC Steckdose Typ 2,22.0,NaN,0,0,NaN


In [17]:
#Seem to be geniue duplicates, therefore, dropping them. 
eautos = eautos.drop_duplicates()

In [18]:
eautos

,operator,street,haus_no,address_extension,postcode,city,federal_land,county,latitude,longitude,commissioning_date,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],public_key1,type_of_plug_2,p2_[kw],public_key2,type_of_plug_3,p3_[kw],public_key3,type_of_plug_4,p4_[kw],public_key4
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,2020-01-11,22.0,normal,2,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,0,0,NaN,0,0,NaN
1,smopi®,Albstraße,14,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,2022-11-23,22.0,normal,4,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22.0,NaN,AC Steckdose Typ 2,22,NaN
2,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,0,NaN,72589,Westerheim,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.510500,9.609000,2019-01-07,22.0,normal,2,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,0,0,NaN,0,0,NaN
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,NaN,73340,Amstetten,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,2019-03-19,98.0,fast,2,AC Kupplung Typ 2,43,NaN,"DC Kupplung Combo, DC CHAdeMO",50,NaN,0,0,NaN,0,0,NaN
4,SWU Energie GmbH,Sterngasse,6,NaN,88481,Balzheim,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,2022-05-01,22.0,normal,1,AC Steckdose Typ 2,22,NaN,0,0,NaN,0,0,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44222,HaloWB,Würzburger Straße,9,NaN,99510,Apolda,Thüringen,Landkreis Weimarer Land,51.013750,11.517313,2023-01-01,11.0,normal,1,AC Kupplung Typ 2,11,NaN,0,0,NaN,0,0,NaN,0,0,NaN
44223,TEAG Mobil GmbH,Dorfstraße,101,NaN,99510,Obertrebra,Thüringen,Landkreis Weimarer Land,51.060798,11.565503,2022-09-12,300.0,fast,2,DC Kupplung Combo,150,NaN,DC Kupplung Combo,150,NaN,0,0,NaN,0,0,NaN
44224,bft Tankstelle Christian Hollmotz,Dorfstr.,101,NaN,99510,Obertrebra,Thüringen,Landkreis Weimarer Land,51.060770,11.565560,2022-01-01,300.0,fast,2,DC Kupplung Combo,300,NaN,DC Kupplung Combo,300,NaN,0,0,NaN,0,0,NaN
44225,Stadtwerke Weimar Stadtversorgungs-GmbH,Wunderwaldstraße,2a,NaN,99518,Bad Sulza,Thüringen,Landkreis Weimarer Land,51.092527,11.628574,2018-08-13,44.0,normal,2,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,0,0,NaN,0,0,NaN


### Data exploration 

#### Checking for values with leading or trailing spaces

In [19]:
eautos.city[eautos.city.str.startswith(' ') | eautos.city.str.endswith(' ')].unique()

array(['Wannweil ', 'Heidelberg ', 'Mannheim ', 'Augsburg ',
       'Berg / Neumarkt ', 'Schöneiche ', 'Frankfurt ', 'Lübstorf ',
       'Bad Salzdetfurth ', 'Hannover ', 'Lehrte ', 'Stadthagen ',
       'Gelsenkirchen ', 'Herne ', 'Köln ', 'Trier ', 'Bingen ',
       'Halle/Saale ', 'Sylt OT Tinnum '], dtype=object)

In [20]:
eautos_copy = eautos.copy()

In [21]:
# Remove leading and trailing spaces from all the object-type columns (including the above) 
columns_object = eautos.select_dtypes(include='object').columns

for column in columns_object:
    eautos[column] = eautos[column].fillna("").str.strip()

In [22]:
eautos

,operator,street,haus_no,address_extension,postcode,city,federal_land,county,latitude,longitude,commissioning_date,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],public_key1,type_of_plug_2,p2_[kw],public_key2,type_of_plug_3,p3_[kw],public_key3,type_of_plug_4,p4_[kw],public_key4
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,NaN,,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,2020-01-11,22.0,normal,2,AC Steckdose Typ 2,NaN,,AC Steckdose Typ 2,NaN,,0,0,,0,0,
1,smopi®,Albstraße,14,,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,2022-11-23,22.0,normal,4,AC Steckdose Typ 2,NaN,,AC Steckdose Typ 2,NaN,,AC Steckdose Typ 2,NaN,,AC Steckdose Typ 2,NaN,
2,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,NaN,,72589,Westerheim,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.510500,9.609000,2019-01-07,22.0,normal,2,AC Steckdose Typ 2,NaN,,AC Steckdose Typ 2,NaN,,0,0,,0,0,
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,,73340,Amstetten,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,2019-03-19,98.0,fast,2,AC Kupplung Typ 2,NaN,,"DC Kupplung Combo, DC CHAdeMO",NaN,,0,0,,0,0,
4,SWU Energie GmbH,Sterngasse,6,,88481,Balzheim,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,2022-05-01,22.0,normal,1,AC Steckdose Typ 2,NaN,,0,0,,0,0,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44222,HaloWB,Würzburger Straße,9,,99510,Apolda,Thüringen,Landkreis Weimarer Land,51.013750,11.517313,2023-01-01,11.0,normal,1,AC Kupplung Typ 2,NaN,,0,0,,0,0,,0,0,
44223,TEAG Mobil GmbH,Dorfstraße,101,,99510,Obertrebra,Thüringen,Landkreis Weimarer Land,51.060798,11.565503,2022-09-12,300.0,fast,2,DC Kupplung Combo,NaN,,DC Kupplung Combo,NaN,,0,0,,0,0,
44224,bft Tankstelle Christian Hollmotz,Dorfstr.,NaN,,99510,Obertrebra,Thüringen,Landkreis Weimarer Land,51.060770,11.565560,2022-01-01,300.0,fast,2,DC Kupplung Combo,NaN,,DC Kupplung Combo,NaN,,0,0,,0,0,
44225,Stadtwerke Weimar Stadtversorgungs-GmbH,Wunderwaldstraße,2a,,99518,Bad Sulza,Thüringen,Landkreis Weimarer Land,51.092527,11.628574,2018-08-13,44.0,normal,2,AC Steckdose Typ 2,NaN,,AC Steckdose Typ 2,NaN,,0,0,,0,0,


#### Checking the NaN values in the entire dataset

In [23]:
eautos.isnull().sum()

operator                         0
street                           0
haus_no                       9646
address_extension              525
postcode                         0
city                             0
federal_land                     0
county                           0
latitude                         0
longitude                        0
commissioning_date               0
power_connection_[kw]            0
type_of_charger                  0
number_of_charging_points        0
type_of_plug_1                   0
p1_[kw]                      38385
public_key1                      2
type_of_plug_2                   0
p2_[kw]                      32261
public_key2                      2
type_of_plug_3                   0
p3_[kw]                       2374
public_key3                      0
type_of_plug_4                   0
p4_[kw]                       1212
public_key4                      0
dtype: int64

### Coulmns with many NaNs

#### "Address_extension"

In [24]:
eautos.address_extension.value_counts(dropna=False)

                                                                                      33559
NaN                                                                                     525
Parkplatz                                                                               176
LADE                                                                                    151
ggü.                                                                                     88
                                                                                      ...  
bei Nr. 19                                                                                1
Ober-Mumbach                                                                              1
An der Aral Tankstelle in die Schulstrasse abbiegen und dann auf der linken Seite.        1
Berliner Str. / Ecke Wasserwerkstr. 2                                                     1
Parkplatz Schillerbuche                                                         

In [25]:
#There is very little useful information in this column, so it was decided to drop the column.
eautos = eautos.drop('address_extension', axis = 1)

#### "Public_key 1, 2 ,3 and 4" columns

In [26]:
eautos.public_key1.value_counts(dropna=False)

                                                                                                                                                                                          33524
03DE06 17AC28 46EA4E 03F38D AC4B8F 728BEC 79EA1F 776E05 EB88C1 6BBBE2 68CA91                                                                                                                  9
3059301306072a8648ce3d020106082a8648ce3d0301070342000465e67f0240cfcfe97dd956062215363c64ac0f401dcd70614d732843ae2f60343b518b5ddc4907a5c1e425a290a44ff52b3c57728d01b41231585cb6eb223298        4
3059301306072a8648ce3d020106082a8648ce3d030107034200049ef1d08c7e7973025b1baf36ffa9930f52002aee81081e7af992c933234a455494fb7365af57a4dce80907c14202eb78fc4bae7362001d31b09e20c243ee826c        3
3059301306072a8648ce3d020106082a8648ce3d03010703420004425d57beb1eb0c91e7305dc55252b719ad926bec7b60e7e8249a3ba3d7b46d79b17d4d06f0555ec3544752671437a606df41b877964b54a7243ec243cdae68f5        3
                                        

In [27]:
# Public keys are strings of numbers that allow the user to check readings remotely. Since they do not provide useful 
# information for our analysis, we will omit these columns.
eautos.drop(columns=['public_key1', 'public_key2', 'public_key3', 'public_key4'], inplace=True)

In [28]:
eautos

,operator,street,haus_no,postcode,city,federal_land,county,latitude,longitude,commissioning_date,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],type_of_plug_2,p2_[kw],type_of_plug_3,p3_[kw],type_of_plug_4,p4_[kw]
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,2020-01-11,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0
1,smopi®,Albstraße,14,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,2022-11-23,22.0,normal,4,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN
2,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,NaN,72589,Westerheim,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.510500,9.609000,2019-01-07,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,73340,Amstetten,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,2019-03-19,98.0,fast,2,AC Kupplung Typ 2,NaN,"DC Kupplung Combo, DC CHAdeMO",NaN,0,0,0,0
4,SWU Energie GmbH,Sterngasse,6,88481,Balzheim,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,2022-05-01,22.0,normal,1,AC Steckdose Typ 2,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44222,HaloWB,Würzburger Straße,9,99510,Apolda,Thüringen,Landkreis Weimarer Land,51.013750,11.517313,2023-01-01,11.0,normal,1,AC Kupplung Typ 2,NaN,0,0,0,0,0,0
44223,TEAG Mobil GmbH,Dorfstraße,101,99510,Obertrebra,Thüringen,Landkreis Weimarer Land,51.060798,11.565503,2022-09-12,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0
44224,bft Tankstelle Christian Hollmotz,Dorfstr.,NaN,99510,Obertrebra,Thüringen,Landkreis Weimarer Land,51.060770,11.565560,2022-01-01,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0
44225,Stadtwerke Weimar Stadtversorgungs-GmbH,Wunderwaldstraße,2a,99518,Bad Sulza,Thüringen,Landkreis Weimarer Land,51.092527,11.628574,2018-08-13,44.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0


### Columns with NO NAN values

#### 'City' column

In [29]:
top_50_cities = eautos['city'].value_counts().head(50).index.tolist()  # Get the top 50 cities with largest number of charging stations


In [30]:
# checking if there are any similar names in the column city(check for potential typos)
#!pip install fuzzywuzzy
#from fuzzywuzzy import fuzz
#from fuzzywuzzy import process

In [31]:
eautos['city'].value_counts(10)

München                             0.037249
Berlin                              0.034776
Hamburg                             0.025353
Stuttgart                           0.018638
Hannover                            0.012599
                                      ...   
Jüterborg                           0.000026
Jüterbog                            0.000026
Niederer Fläming                    0.000026
Niederer Fläming OT Hohenseefeld    0.000026
Großheringen                        0.000026
Name: city, Length: 6026, dtype: float64

In [32]:
cities = eautos['city'].value_counts()
cities.head(50)

München                1431
Berlin                 1336
Hamburg                 974
Stuttgart               716
Hannover                484
Dortmund                469
Düsseldorf              437
Rüsselsheim am Main     411
Sindelfingen            330
Köln                    312
Regensburg              310
Essen                   284
Wolfsburg               279
Dresden                 272
Leipzig                 253
Bremen                  249
Dingolfing              221
Nürnberg                184
Bonn                    178
Chemnitz                147
Aachen                  134
Mannheim                131
Bochum                  129
Bielefeld               124
Osnabrück               118
Augsburg                117
Zwickau                 115
Heilbronn               109
Frankfurt am Main       109
Ingolstadt              108
Wiesbaden               105
Unterschleißheim        102
Paderborn               101
Heidelberg               99
Braunschweig             96
Baunatal            

Using str.contains method to identify spelling differences 

In [33]:
# Checking for spelling/name variations in the 50 largest cities
cities_series = pd.Series(eautos['city'].unique())
for city in top_50_cities:
    city_variations = cities_series[cities_series.str.contains(city)].unique()
    print("City:", city)
    print("Variations:", city_variations)
    print()

City: München
Variations: ['München' 'München-Laim' 'München Nord' 'München-Freiham'
 'München - Neufriedenheim' 'München-Mittelsendling' 'München Sendling'
 'München-Neuperlach' 'Grafing bei München' 'München Flughafen'
 'München-Flughafen' 'Puchheim b. München' 'Kirchheim bei München'
 'Garching b. München' 'Garching bei München']

City: Berlin
Variations: ['Berlin' 'Berlin-Pankow' 'Berlin,' 'Berlin (Tempelhof)'
 'Berlin Osterpreussen' 'Berlin-Lichterfelde' 'Berlin Marienfelde'
 'Berlin-Köpenick' 'Berlin-Marzahn-Hellersdorf' 'Berlin Marzahn'
 'Berlin-Alt-Hohenschönhausen' 'Berlin-Friedrichsfelde'
 'Berlin-Reinickendorf' 'Berlin-Spandau' 'Bernau bei Berlin'
 'Berlin Hönow' 'Neuenhagen bei Berlin' 'Rüdersdorf bei Berlin']

City: Hamburg
Variations: ['Hamburg' 'Hamburg-Wilhelmsburg' 'Hamburg-Wandsbeck' 'Hamburg-Duvenstedt'
 'Hamburg-Eidelstedt' 'Hamburg/ Moorfleet' 'Wentorf bei Hamburg']

City: Stuttgart
Variations: ['Stuttgart' 'Stuttgart-Mitte' 'Stuttgart-Obertürkheim'
 'Stuttgart-Müh

In [34]:
eautos[(eautos['county'] == 'Kreisfreie Stadt München') & (eautos['city'] != 'München')]['city'].unique()

array(['München-Laim', 'München Nord', 'Munich', 'München-Freiham',
       'München - Neufriedenheim', 'München-Mittelsendling',
       'München Sendling', 'München-Neuperlach'], dtype=object)

In [35]:
eautos[(eautos['county'] == 'Kreisfreie Stadt Berlin') & (eautos['city'] != 'Berlin')]['city'].unique()

array(['Belin', 'Essen', 'Berlin-Pankow', 'Berlin,',
       'Tempelhof-Schöneberg', 'Berlin (Tempelhof)',
       'Berlin Osterpreussen', 'Berlin-Lichterfelde',
       'Berlin Marienfelde', 'berlin', 'Berlin-Köpenick',
       'Berlin-Marzahn-Hellersdorf', 'Berlin Marzahn',
       'Berlin-Alt-Hohenschönhausen', 'Berlin-Friedrichsfelde',
       'Berlin-Reinickendorf', 'Berlin-Spandau'], dtype=object)

In [36]:
eautos[(eautos['county'] == 'Kreisfreie Stadt Hamburg') & (eautos['city'] != 'Hamburg')]['city'].unique()

array(['Hamurg', 'Mitte', 'Hamburg-Wilhelmsburg', 'Hamburg-Wandsbeck',
       'Hamburg-Duvenstedt', 'Hamburg-Eidelstedt', 'Hamburg/ Moorfleet'],
      dtype=object)

In [37]:
eautos[(eautos['county'] == 'Kreisfreie Stadt Wiesbaden') & (eautos['city'] != 'Wiesbaden')]['city'].unique()

array(['Mainz-Kastel', 'Wiesbaden-Biebrich', 'Wiesbaden-Nordenstadt'],
      dtype=object)

In [38]:
eautos[(eautos['county'] == 'Kreisfreie Stadt Osnabrück') & (eautos['city'] != 'Osnabrück')]['city'].unique()

array(['Osnbrück'], dtype=object)

In [39]:
eautos[(eautos['county'] == 'Stadtkreis Stuttgart') & (eautos['city'] != 'Stuttgart')]['city'].unique()

array(['Stuttgart-Mitte', 'Stuttgart-Obertürkheim', 'Bad Canstatt',
       'Stuttgart-Mühlhausen', 'Stuttgart-Feuerbach', 'Sttutgart',
       'Stuttgart-Möhringen'], dtype=object)

In [40]:
eautos[(eautos['county'] == 'Kreisfreie Stadt Frankfurt Ulm') & (eautos['city'] != 'Ulm')]['city'].unique()

array([], dtype=object)

In [41]:
eautos[(eautos['county'] == 'Stadtkreis Heidelberg') & (eautos['city'] != 'Heidelberg')]['city'].unique() 

array(['heidelberg', 'HEidelberg'], dtype=object)

In [42]:
import pandas as pd

# Assuming 'eautos' is the DataFrame containing the city column

# Define the dictionary of cities and their variations
cities_corrections = {
    'München': ['München', 'München-Laim', 'Munich', 'München Nord', 'München-Freiham', 'München - Neufriedenheim', 'München-Mittelsendling', 'München Sendling', 'München-Neuperlach', 'Grafing bei München', 'München Flughafen', 'München-Flughafen'],
    'Berlin': ['Berlin', 'Tempelhof-Schöneberg', 'Belin', 'berlin', 'Berlin-Pankow', 'Berlin,', 'Berlin (Tempelhof)', 'Berlin Osterpreussen', 'Berlin-Lichterfelde', 'Berlin Marienfelde', 'Berlin-Köpenick', 'Berlin-Marzahn-Hellersdorf', 'Berlin Marzahn', 'Berlin-Alt-Hohenschönhausen', 'Berlin-Friedrichsfelde', 'Berlin-Reinickendorf', 'Berlin-Spandau', 'Bernau bei Berlin', 'Berlin Hönow', 'Neuenhagen bei Berlin', 'Rüdersdorf bei Berlin'],
    'Hamburg': ['Hamburg', 'Mitte', 'Hamurg', 'Hamburg-Wilhelmsburg', 'Hamburg-Wandsbeck', 'Hamburg-Duvenstedt', 'Hamburg-Eidelstedt', 'Hamburg/ Moorfleet', 'Wentorf bei Hamburg'],
    'Stuttgart': ['Stuttgart', 'Sttutgart', 'Stuttgart-Mitte', 'Stuttgart-Obertürkheim', 'Stuttgart-Mühlhausen', 'Stuttgart-Feuerbach', 'Stuttgart-Möhringen'],
    'Hannover': ['Hannover', 'Hannover-Vahrenwald', 'Hannover'],
    'Dortmund': ['Dortmund', 'Dortmund-Mitte'],
    'Osnabrück': ['Osnabrück', 'Osnbrück'],
    'Köln': ['Köln', 'Köln Deutz', 'Köln-Niehl', 'Köln-Pesch', 'Köln Worringen', 'Köln', 'Köln Ehrenfeld', 'Köln Braunsfeld', 'Köln Ossendorf', 'Köln-Poll', 'Köln-Merheim', 'Köln-Porz-Eil', 'Köln Porz'],
    'Regensburg': ['Regensburg', 'Regensburg-Schwabelweis', 'Regensburg-Ost'],
    'Essen': ['Essen', 'Essen (Oldenburg)', 'Addrup-Essen/Oldb.', 'Essen-Kettwig', 'Essen-Altenessen'],
    'Dresden': ['Dresden', 'Dresden-Hellerau', 'Dresden-Radebeul', 'Dresden Gabitz', 'Dresden-Prohlis', 'Dresden Striesen'],
    'Leipzig': ['Leipzig', 'Leipzig-Eutritzsch', 'Leipzig-Gohlis', 'Leipzig - Alt-West', 'Leipzig-Plagwitz', 'Leipzig Großzschocher', 'Leipzig-Seehausen'],
    'Bremen': ['Bremen', 'Bremen-Vahr', 'Achim bei Bremen'],
    'Nürnberg': ['Nürnberg', 'Nürnberg/Moorenbrunn'],
    'Bonn': ['Bonndorf im Schwarzwald', 'Bonndorf', 'Bonn', 'Bonn-Nordstadt'],
    'Chemnitz': ['Chemnitz', 'Chemnitz-Helbersdorf', 'Chemnitz/Grüna', 'Chemnitz-Röhrsdorf'],
    'Aachen': ['Aachen', 'Aachen-Königshügel', 'Aachen-Brand'],
    'Bochum': ['Bochum', 'Bochum-Hiltrop', 'Bochum-Eppendorf'],
    'Bielefeld': ['Bielefeld', 'Bielefeld-Oldentrup'],
    'Zwickau': ['Zwickau', 'Zwickau OT Crossen'],
    'Wiesbaden': ['Wiesbaden', 'Wiesbaden-Biebrich', 'Wiesbaden-Nordenstadt', 'Mainz-Kastel'],
    'Münster': ['Münstertal', 'Münstertal/Schwarzwald', 'Münster', 'Münsterschwarzach', 'Bad Münstereifel', 'Münster-Hiltrup', 'Münstermaifeld'],
    'Fürth': ['Fürth', 'Fürthen'],
    'Oberhausen': ['Oberhausen-Rheinhausen', 'Oberhausen', 'Oberhausen-Sterkrade'],
    'Karlsruhe': ['Karlsruhe', 'Karlsruhe-Neureut', 'Karlsruhe-Grötzingen'],
    'Darmstadt': ['Darmstadt', 'Darmstadt-Eberstadt', 'Wixhausen Darmstadt'],
    'Kiel': ['Kiel', 'Kiel-Südfriedhof', 'Kiel-Elmschenhagen'],
    'Heidelberg': ['Heidelberg', 'heidelberg', 'HEidelberg'],
    'Ulm': ['Lichtenau-Ulm', 'Ulm - Eggingen', 'Ulm', 'Ulm-Unterweiler', 'Ulm-Jungingen', 'Ulm Jungingen', 'Neu-Ulm', 'Neu Ulm', 'Ulmen'],
}

In [43]:
for original_city, variations in cities_corrections.items():
    eautos['city'] = eautos['city'].replace(variations, original_city)

# Verifing the updated 'city' column
eautos['city']

0         Heroldstatt
1         Heroldstatt
2          Westerheim
3           Amstetten
4            Balzheim
             ...     
44222          Apolda
44223      Obertrebra
44224      Obertrebra
44225       Bad Sulza
44226    Großheringen
Name: city, Length: 38417, dtype: object

#### More work with wrong city names

In [44]:
cities = eautos.city
cities[cities.str.contains('Frankfurt')].unique()

array(['Frankfurt (Oder)', 'Frankfurt/Oder', 'Frankfurt am Main',
       'Frankfurt', 'Frankfurt-Fechenheim', 'Frankfurt-Nieder-Eschbach',
       'Frankfurt-Niederrad'], dtype=object)

Frankfurt has more varations. Replacing the variations accordingly. 

In [45]:
frankfurts_modification = {
    'Frankfurt am Main': ['Frankfurt', 'Frankfurt-Fechenheim', 'Frankfurt-Nieder-Eschbach', 'Frankfurt-Niederrad'],
    'Frankfurt (Oder)': ['Frankfurt/Oder']
}

In [46]:
for original_city, variations in frankfurts_modification.items():
    eautos['city'] = eautos['city'].replace(variations, original_city)

In [47]:
# double-checking
eautos.city[eautos.city.str.contains('Frankfurt')].unique()

array(['Frankfurt (Oder)', 'Frankfurt am Main'], dtype=object)

In [48]:
eautos.city.value_counts().head(50)

München                1444
Berlin                 1363
Hamburg                 982
Stuttgart               724
Hannover                485
Dortmund                470
Düsseldorf              437
Rüsselsheim am Main     411
Köln                    343
Sindelfingen            330
Regensburg              312
Essen                   292
Wolfsburg               279
Dresden                 278
Leipzig                 259
Bremen                  251
Dingolfing              221
Bonn                    186
Nürnberg                185
Frankfurt am Main       170
Chemnitz                150
Aachen                  136
Ulm                     135
Mannheim                131
Bochum                  131
Bielefeld               125
Osnabrück               119
Augsburg                117
Zwickau                 116
Heilbronn               109
Wiesbaden               109
Ingolstadt              108
Münster                 104
Unterschleißheim        102
Paderborn               101
Heidelberg          

In [49]:
eautos.county.value_counts()

Kreisfreie Stadt München              1439
Kreisfreie Stadt Berlin               1357
Kreisfreie Stadt Hamburg               981
Stadtkreis Stuttgart                   725
Landkreis Region Hannover              703
                                      ... 
Landkreis Südwestpfalz                  14
Landkreis Birkenfeld                    13
Kreisfreie Stadt Amberg                 12
Kreisfreie Stadt Offenbach am Main      10
Kreisfreie Stadt Schwabach               9
Name: county, Length: 401, dtype: int64

In [50]:
top_50_cities = eautos['city'].value_counts().head(50).index.tolist()

###### As there are still too many values in the cities column, i would like to replace the cities that are not in the top 50 list with the value "other + the name of the federal land they are located 

In [51]:
top_50_cities = eautos['city'].value_counts().head(50).index.tolist()

# Creating a mask to identify cities not in the top-50 list
mask = ~eautos['city'].isin(top_50_cities)

# Assign 'other + federal land' to cities not in the top-50 list
eautos.loc[mask, 'city'] = 'other ' + eautos.loc[mask, 'county']

eautos

,operator,street,haus_no,postcode,city,federal_land,county,latitude,longitude,commissioning_date,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],type_of_plug_2,p2_[kw],type_of_plug_3,p3_[kw],type_of_plug_4,p4_[kw]
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,NaN,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,2020-01-11,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0
1,smopi®,Albstraße,14,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,2022-11-23,22.0,normal,4,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN
2,Albwerk GmbH & Co. KG,Parkplatz Campingplatz,NaN,72589,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.510500,9.609000,2019-01-07,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,73340,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,2019-03-19,98.0,fast,2,AC Kupplung Typ 2,NaN,"DC Kupplung Combo, DC CHAdeMO",NaN,0,0,0,0
4,SWU Energie GmbH,Sterngasse,6,88481,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,2022-05-01,22.0,normal,1,AC Steckdose Typ 2,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44222,HaloWB,Würzburger Straße,9,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.013750,11.517313,2023-01-01,11.0,normal,1,AC Kupplung Typ 2,NaN,0,0,0,0,0,0
44223,TEAG Mobil GmbH,Dorfstraße,101,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060798,11.565503,2022-09-12,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0
44224,bft Tankstelle Christian Hollmotz,Dorfstr.,NaN,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060770,11.565560,2022-01-01,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0
44225,Stadtwerke Weimar Stadtversorgungs-GmbH,Wunderwaldstraße,2a,99518,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.092527,11.628574,2018-08-13,44.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0


In [52]:
eautos['city'].value_counts()

München                                     1444
Berlin                                      1363
Hamburg                                      982
Stuttgart                                    724
Hannover                                     485
                                            ... 
other Stadtkreis Ulm                           1
other Stadtkreis Stuttgart                     1
other Kreisfreie Stadt Frankfurt am Main       1
other Kreisfreie Stadt Wolfsburg               1
other Kreisfreie Stadt Kiel                    1
Name: city, Length: 424, dtype: int64

In [53]:
eautos.loc[eautos['city'].str.startswith('other Kreisfreie Stadt'), 'city'] = eautos.loc[eautos['city'].str.startswith('other Kreisfreie Stadt'), 'city'].str.split().str[-1]

In [54]:
eautos['city'].value_counts().head(10)

München                    1444
Berlin                     1363
Hamburg                     982
Stuttgart                   724
Hannover                    485
Dortmund                    470
Düsseldorf                  437
Rüsselsheim am Main         411
other Landkreis München     357
Köln                        345
Name: city, dtype: int64

#### "Federal_land" column

In [55]:
eautos['federal_land'].value_counts()

Bayern                    8058
Nordrhein-Westfalen       7029
Baden-Württemberg         6512
Niedersachsen             3795
Hessen                    2908
Sachsen                   1613
Rheinland-Pfalz           1482
Schleswig-Holstein        1435
Berlin                    1357
Hamburg                    981
Brandenburg                806
Thüringen                  729
Sachsen-Anhalt             616
Mecklenburg-Vorpommern     493
Saarland                   321
Bremen                     282
Name: federal_land, dtype: int64

#### 'Operator' column

In [56]:
eautos.operator.value_counts()

EnBW mobility+ AG und Co.KG          2045
E.ON Drive GmbH                      1999
EWE Go GmbH                           922
Mercedes-Benz AG                      825
Westenergie Metering GmbH             798
                                     ... 
Jörg O. Richter e.K.                    1
Taxi Hagemann                           1
Kurierdienst                            1
Siegfried Erkner & Sohn GmbH            1
bft Tankstelle Christian Hollmotz       1
Name: operator, Length: 4903, dtype: int64

In [57]:
eautos['operator'].value_counts(dropna = False).head(50)

EnBW mobility+ AG und Co.KG                                              2045
E.ON Drive GmbH                                                          1999
EWE Go GmbH                                                               922
Mercedes-Benz AG                                                          825
Westenergie Metering GmbH                                                 798
Stromnetz Hamburg GmbH                                                    709
Privatperson                                                              639
Allego GmbH                                                               625
SWM Versorgungs GmbH                                                      598
ALDI SÜD                                                                  563
Berliner Stadtwerke KommunalPartner GmbH                                  526
Mer Germany GmbH                                                          488
Lidl                                                            

It seems to be a few large operators, therefore there is a need to check how many operators we have. I have chosed to see how many operators have less than 50 charging stations.

In [58]:
operator_counts = eautos['operator'].value_counts()
operators_less_than_100 = operator_counts[operator_counts < 50]
operators_less_than_100

badenova AG & Co. KG                   49
Albwerk GmbH & Co. KG                  49
EMobility Netzwerk Deutschland GmbH    48
Stadtwerke Bamberg                     47
FairEnergie GmbH                       47
                                       ..
Jörg O. Richter e.K.                    1
Taxi Hagemann                           1
Kurierdienst                            1
Siegfried Erkner & Sohn GmbH            1
bft Tankstelle Christian Hollmotz       1
Name: operator, Length: 4799, dtype: int64

As grouping by federal state supersedes most of the true operators, it has been decided to group those within the 50 cities with the highest number of chargers under "other + city name", otherwise "other + federal state name".

In [59]:
# 'operator_counts = eautos['operator'].value_counts()
# operators_less_than_100 = operator_counts[operator_counts < 00].index.tolist()

# # Creating a mask to identify operators with less than 50 occurrences
# mask = eautos['operator'].isin(operators_less_than_100) & ~eautos['city'].isin(top_50_cities)

# # Assign 'other and federal_land' to operators with less than 50 occurrences and city not in the top-50 list
# eautos.loc[mask, 'operator'] = 'other ' + eautos.loc[mask, 'federal_land']

# # Assign 'other + city' to operators with less than 50 occurrences and city in the top-50 list
# eautos.loc[mask == False, 'operator'] = 'other ' + eautos.loc[mask == False, 'city']''

In [60]:
top_100_operators = eautos['operator'].value_counts().head(100).index.tolist()

# Creating a mask to identify operators not in the top-50 list
mask = ~eautos['operator'].isin(top_100_operators)

# Assign 'other + county' to operators not in the top-50 list
eautos.loc[mask, 'operator'] = 'other ' + eautos.loc[mask, 'county']

In [61]:
eautos.operator.value_counts()

EnBW mobility+ AG und Co.KG         2045
E.ON Drive GmbH                     1999
EWE Go GmbH                          922
Mercedes-Benz AG                     825
Westenergie Metering GmbH            798
                                    ... 
other Kreisfreie Stadt Schwabach       4
other Landkreis Südwestpfalz           4
other Kreisfreie Stadt Schwerin        3
other Landkreis Kusel                  2
other Landkreis Kulmbach               1
Name: operator, Length: 501, dtype: int64

In [62]:
eautos

,operator,street,haus_no,postcode,city,federal_land,county,latitude,longitude,commissioning_date,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],type_of_plug_2,p2_[kw],type_of_plug_3,p3_[kw],type_of_plug_4,p4_[kw]
0,other Landkreis Alb-Donau-Kreis,Ennabeurer Weg,NaN,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,2020-01-11,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0
1,other Landkreis Alb-Donau-Kreis,Albstraße,14,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,2022-11-23,22.0,normal,4,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN
2,other Landkreis Alb-Donau-Kreis,Parkplatz Campingplatz,NaN,72589,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.510500,9.609000,2019-01-07,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,73340,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,2019-03-19,98.0,fast,2,AC Kupplung Typ 2,NaN,"DC Kupplung Combo, DC CHAdeMO",NaN,0,0,0,0
4,SWU Energie GmbH,Sterngasse,6,88481,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,2022-05-01,22.0,normal,1,AC Steckdose Typ 2,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44222,other Landkreis Weimarer Land,Würzburger Straße,9,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.013750,11.517313,2023-01-01,11.0,normal,1,AC Kupplung Typ 2,NaN,0,0,0,0,0,0
44223,TEAG Mobil GmbH,Dorfstraße,101,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060798,11.565503,2022-09-12,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0
44224,other Landkreis Weimarer Land,Dorfstr.,NaN,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060770,11.565560,2022-01-01,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0
44225,other Landkreis Weimarer Land,Wunderwaldstraße,2a,99518,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.092527,11.628574,2018-08-13,44.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0


### Repalcing all remaining NaNs 

In [63]:
eautos.isna().sum()

operator                         0
street                           0
haus_no                       9646
postcode                         0
city                             0
federal_land                     0
county                           0
latitude                         0
longitude                        0
commissioning_date               0
power_connection_[kw]            0
type_of_charger                  0
number_of_charging_points        0
type_of_plug_1                   0
p1_[kw]                      38385
type_of_plug_2                   0
p2_[kw]                      32261
type_of_plug_3                   0
p3_[kw]                       2374
type_of_plug_4                   0
p4_[kw]                       1212
dtype: int64

## Commissioning_date column

In [64]:
eautos.commissioning_date.dtype

dtype('<M8[ns]')

In [65]:
eautos['commissioning_year'] = eautos['commissioning_date'].dt.year
eautos['commissioning_month'] = eautos['commissioning_date'].dt.strftime('%B')

In [66]:
eautos

,operator,street,haus_no,postcode,city,federal_land,county,latitude,longitude,commissioning_date,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],type_of_plug_2,p2_[kw],type_of_plug_3,p3_[kw],type_of_plug_4,p4_[kw],commissioning_year,commissioning_month
0,other Landkreis Alb-Donau-Kreis,Ennabeurer Weg,NaN,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,2020-01-11,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2020,January
1,other Landkreis Alb-Donau-Kreis,Albstraße,14,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,2022-11-23,22.0,normal,4,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,2022,November
2,other Landkreis Alb-Donau-Kreis,Parkplatz Campingplatz,NaN,72589,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.510500,9.609000,2019-01-07,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2019,January
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,73340,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,2019-03-19,98.0,fast,2,AC Kupplung Typ 2,NaN,"DC Kupplung Combo, DC CHAdeMO",NaN,0,0,0,0,2019,March
4,SWU Energie GmbH,Sterngasse,6,88481,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,2022-05-01,22.0,normal,1,AC Steckdose Typ 2,NaN,0,0,0,0,0,0,2022,May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44222,other Landkreis Weimarer Land,Würzburger Straße,9,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.013750,11.517313,2023-01-01,11.0,normal,1,AC Kupplung Typ 2,NaN,0,0,0,0,0,0,2023,January
44223,TEAG Mobil GmbH,Dorfstraße,101,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060798,11.565503,2022-09-12,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0,2022,September
44224,other Landkreis Weimarer Land,Dorfstr.,NaN,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060770,11.565560,2022-01-01,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0,2022,January
44225,other Landkreis Weimarer Land,Wunderwaldstraße,2a,99518,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.092527,11.628574,2018-08-13,44.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2018,August


In [67]:
eautos = eautos.drop('commissioning_date', axis=1)

In [68]:
eautos

,operator,street,haus_no,postcode,city,federal_land,county,latitude,longitude,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],type_of_plug_2,p2_[kw],type_of_plug_3,p3_[kw],type_of_plug_4,p4_[kw],commissioning_year,commissioning_month
0,other Landkreis Alb-Donau-Kreis,Ennabeurer Weg,NaN,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2020,January
1,other Landkreis Alb-Donau-Kreis,Albstraße,14,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,22.0,normal,4,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,2022,November
2,other Landkreis Alb-Donau-Kreis,Parkplatz Campingplatz,NaN,72589,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.510500,9.609000,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2019,January
3,EnBW mobility+ AG und Co.KG,Hauptstraße,91c,73340,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,98.0,fast,2,AC Kupplung Typ 2,NaN,"DC Kupplung Combo, DC CHAdeMO",NaN,0,0,0,0,2019,March
4,SWU Energie GmbH,Sterngasse,6,88481,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,22.0,normal,1,AC Steckdose Typ 2,NaN,0,0,0,0,0,0,2022,May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44222,other Landkreis Weimarer Land,Würzburger Straße,9,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.013750,11.517313,11.0,normal,1,AC Kupplung Typ 2,NaN,0,0,0,0,0,0,2023,January
44223,TEAG Mobil GmbH,Dorfstraße,101,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060798,11.565503,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0,2022,September
44224,other Landkreis Weimarer Land,Dorfstr.,NaN,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060770,11.565560,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0,2022,January
44225,other Landkreis Weimarer Land,Wunderwaldstraße,2a,99518,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.092527,11.628574,44.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2018,August


In [69]:
eautos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38417 entries, 0 to 44226
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   operator                   38417 non-null  object 
 1   street                     38417 non-null  object 
 2   haus_no                    28771 non-null  object 
 3   postcode                   38417 non-null  int64  
 4   city                       38417 non-null  object 
 5   federal_land               38417 non-null  object 
 6   county                     38417 non-null  object 
 7   latitude                   38417 non-null  float64
 8   longitude                  38417 non-null  float64
 9   power_connection_[kw]      38417 non-null  float64
 10  type_of_charger            38417 non-null  object 
 11  number_of_charging_points  38417 non-null  int64  
 12  type_of_plug_1             38417 non-null  object 
 13  p1_[kw]                    32 non-null     obj

### Addressing the duplcated postcodes

In [70]:
duplicates = eautos['postcode'].duplicated(keep=False)
duplicate_rows = eautos[duplicates]
duplicate_postcodes_df = pd.DataFrame(duplicate_rows)
duplicate_postcodes_df

,operator,street,haus_no,postcode,city,federal_land,county,latitude,longitude,power_connection_[kw],type_of_charger,number_of_charging_points,type_of_plug_1,p1_[kw],type_of_plug_2,p2_[kw],type_of_plug_3,p3_[kw],type_of_plug_4,p4_[kw],commissioning_year,commissioning_month
0,other Landkreis Alb-Donau-Kreis,Ennabeurer Weg,NaN,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.442398,9.659075,22.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2020,January
1,other Landkreis Alb-Donau-Kreis,Albstraße,14,72535,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.449353,9.672201,22.0,normal,4,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,2022,November
5,SWU Energie GmbH,Herwigstraße,21,89129,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.485240,10.028291,44.0,normal,1,AC Steckdose Typ 2,NaN,0,0,0,0,0,0,2020,October
6,EnBW mobility+ AG und Co.KG,Flurstück,313,89129,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.456817,10.029649,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0,2019,March
8,EnBW mobility+ AG und Co.KG,Flurstück,313,89129,other Landkreis Alb-Donau-Kreis,Baden-Württemberg,Landkreis Alb-Donau-Kreis,48.456817,10.029649,173.0,fast,2,"DC Kupplung Combo, DC CHAdeMO",NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2019,December
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44222,other Landkreis Weimarer Land,Würzburger Straße,9,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.013750,11.517313,11.0,normal,1,AC Kupplung Typ 2,NaN,0,0,0,0,0,0,2023,January
44223,TEAG Mobil GmbH,Dorfstraße,101,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060798,11.565503,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0,2022,September
44224,other Landkreis Weimarer Land,Dorfstr.,NaN,99510,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.060770,11.565560,300.0,fast,2,DC Kupplung Combo,NaN,DC Kupplung Combo,NaN,0,0,0,0,2022,January
44225,other Landkreis Weimarer Land,Wunderwaldstraße,2a,99518,other Landkreis Weimarer Land,Thüringen,Landkreis Weimarer Land,51.092527,11.628574,44.0,normal,2,AC Steckdose Typ 2,NaN,AC Steckdose Typ 2,NaN,0,0,0,0,2018,August


In [71]:
eautos.to_csv('eautos_clean.csv', index=False)

In [72]:
eautos['federal_land'].value_counts()

Bayern                    8058
Nordrhein-Westfalen       7029
Baden-Württemberg         6512
Niedersachsen             3795
Hessen                    2908
Sachsen                   1613
Rheinland-Pfalz           1482
Schleswig-Holstein        1435
Berlin                    1357
Hamburg                    981
Brandenburg                806
Thüringen                  729
Sachsen-Anhalt             616
Mecklenburg-Vorpommern     493
Saarland                   321
Bremen                     282
Name: federal_land, dtype: int64

In [73]:
eautos['city'].value_counts()

München                       1444
Berlin                        1363
Hamburg                        982
Stuttgart                      724
Hannover                       485
                              ... 
Amberg                          12
Main                            11
Schwabach                        9
other Stadtkreis Ulm             1
other Stadtkreis Stuttgart       1
Name: city, Length: 417, dtype: int64

### Checking the range for fast and normal charging 

In [74]:
eautos['type_of_charger'].value_counts()

normal    32218
fast       6199
Name: type_of_charger, dtype: int64

In [75]:
value_counts_percent = eautos['type_of_charger'].value_counts(normalize=True) * 100

print("Value counts in percent:")
print(value_counts_percent)

Value counts in percent:
normal    83.863914
fast      16.136086
Name: type_of_charger, dtype: float64


## Cars registered a day

In [77]:
additional_ecars_needed = 15000000 - 1951270
number_of_days = (2030-2023) * 365
average_ecars_registered_per_day = additional_ecars_needed / number_of_days

print("Average electric cars registered per day:", average_ecars_registered_per_day)

Average electric cars registered per day: 5107.135029354207


In [78]:
number_of_cars_2018 = 124295
number_of_cars_2023 = 1950000
growth_factor = number_of_cars_2023 / number_of_cars_2018

print("Growth factor:", growth_factor)

Growth factor: 15.688483044370248


### Web scraping

In [79]:
import requests
from bs4 import BeautifulSoup

In [80]:
url = "https://www.motoringelectric.com/charging/difference-between-slow-fast-rapid-charging/#:~:text=Electric%20cars%20require%20a%20direct,speeding%20up%20the%20charging%20time"
response = requests.get(url)

In [81]:
soup = BeautifulSoup(response.content, "html.parser")

In [82]:
#soup

<!DOCTYPE html >

<!--[if IE 8]>    <html class="ie8" lang="en"> <![endif]-->
<!--[if IE 9]>    <html class="ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-US"> <!--<![endif]-->
<head>
<title>What is the difference between slow, fast and rapid EV chargers?</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://www.motoringelectric.com/xmlrpc.php" rel="pingback"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<link href="https://www.motoringelectric.com/wp-content/uploads/2020/09/MR-Battery-Logo-square_152.jpg" rel="apple-touch-icon" sizes="152x152"/>
<!-- This site is optimized with the Yoast SEO Premium plugin v20.9 (Yoast SEO v20.9) - https://yoast.com/wordpress/plugins/seo/ -->
<meta content="Want to understand the difference between slow, fast and rapid charging for electric cars? Our five-minute guide explains all." name="descriptio

In [83]:
lines = soup.find_all('p', text=lambda text: text and "The majority of slow charging points are rated at 3.6kW and will recharge an electric car in eight to 12 hours. This makes them suited to overnight charging, or when you’re in the office for a full day of work" in text)

for line in lines:
    print(line.text)

The majority of slow charging points are rated at 3.6kW and will recharge an electric car in eight to 12 hours. This makes them suited to overnight charging, or when you’re in the office for a full day of work.
